In [110]:
from vassar_feetech_servo_sdk import ServoController, find_servo_port


'''
sudo uhubctl

Current status for hub 1-9 [2109:2813 VIA Labs, Inc. USB2.0 Hub, USB 2.10, 4 ports, ppps]
 Port 1: 0103 power enable connect [1a86:55d3 USB Single Serial 5A7A055855]
 Port 2: 0507 power highspeed suspend enable connect [046d:085c C922 Pro Stream Webcam F657B3BF]
 Port 3: 0103 power enable connect [1a86:55d3 USB Single Serial 5A7A055705]
 Port 4: 0100 power

sudo uhubctl -l 1-9 -p 1 -a cycle
sudo uhubctl -l 1-9 -p 3 -a cycle
'''

try:
    ports = find_servo_port(return_all=True)
    print(f"Available ports: {ports}")
except Exception as e:
    print(f"No ports found: {e}")

SERVO_IDS = [1, 2, 3, 4, 5, 6, 7]
LEADER_SERVO_TYPE = "hls"  # or "hls"
FOLLOWER_SERVO_TYPE = "sts"  # or "hls"
VOLTAGE_THRESHOLD = 9.0  # Volts
ctrl1 = ServoController(SERVO_IDS, LEADER_SERVO_TYPE, ports[0])
ctrl2 = ServoController(SERVO_IDS, FOLLOWER_SERVO_TYPE, ports[1])

with ctrl1, ctrl2:
    voltage1 = ctrl1.read_voltage(SERVO_IDS[0])

    # Leader arm should have voltage ~5V, follower arm ~12V
    if voltage1 < VOLTAGE_THRESHOLD:
        leader, follower = ctrl1, ctrl2
        print(f"{ports[0]}: Leader (voltage: {voltage1:.1f}V)")
        print(f"{ports[1]}: Follower")
    else:
        leader, follower = ctrl2, ctrl1
        print(f"{ports[0]}: Follower (voltage: {voltage1:.1f}V)")
        print(f"{ports[1]}: Leader")


    positions = leader.read_all_positions()
    for motor_id, pos in positions.items():
        print(f"Leader Motor {motor_id}: {pos} ({pos/4095*100:.1f}%)")
    positions = follower.read_all_positions()
    for motor_id, pos in positions.items():
        print(f"Follower Motor {motor_id}: {pos} ({pos/4095*100:.1f}%)")

    leader.set_middle_position()
    follower.set_middle_position()

Available ports: ['/dev/ttyACM1', '/dev/ttyACM2']
Checking servo phase values...
Motor 1: Phase already 0 ✓
Motor 2: Phase already 0 ✓
Motor 3: Phase already 0 ✓
Motor 4: Phase already 0 ✓
Motor 5: Phase already 0 ✓
Motor 6: Phase already 0 ✓
Motor 7: Phase already 0 ✓
Checking servo phase values...
Motor 1: Phase already 0 ✓
Motor 2: Phase already 0 ✓
Motor 3: Phase already 0 ✓
Motor 4: Phase already 0 ✓
Motor 5: Phase already 0 ✓
Motor 6: Phase already 0 ✓
Motor 7: Phase already 0 ✓
/dev/ttyACM1: Leader (voltage: 8.4V)
/dev/ttyACM2: Follower
Leader Motor 1: 2015 (49.2%)
Leader Motor 2: 2723 (66.5%)
Leader Motor 3: 1843 (45.0%)
Leader Motor 4: 1879 (45.9%)
Leader Motor 5: 2057 (50.2%)
Leader Motor 6: 2049 (50.0%)
Leader Motor 7: 2047 (50.0%)
Follower Motor 1: 2013 (49.2%)
Follower Motor 2: 2722 (66.5%)
Follower Motor 3: 1845 (45.1%)
Follower Motor 4: 1879 (45.9%)
Follower Motor 5: 2056 (50.2%)
Follower Motor 6: 2051 (50.1%)
Follower Motor 7: 2051 (50.1%)

Setting middle position using

In [109]:
"""Minimal teleoperation: auto-detect leader/follower by voltage.

Leader arm: voltage < 9V
Follower arm: voltage > 9V
"""

import time
from vassar_feetech_servo_sdk import ServoController, find_servo_port


def main():
    # Configuration
    SERVO_IDS = [1, 2, 3, 4, 5, 6, 7]
    LEADER_SERVO_TYPE = "hls"  # or "hls"
    FOLLOWER_SERVO_TYPE = "sts"  # or "hls"
    VOLTAGE_THRESHOLD = 9.0  # Volts
    FREQUENCY = 200  # Hz

    # Auto-detect ports
    try:
        ports = find_servo_port(return_all=True)
        if len(ports) < 2:
            print(f"Error: Need 2 servo ports but found {len(ports)}")
            if ports:
                print(f"Available ports: {', '.join(ports)}")
            return
    except Exception as e:
        print(f"Error finding servo ports: {e}")
        return

    print(f"Found ports: {ports[0]}, {ports[1]}")

    # Create controllers
    ctrl1 = ServoController(SERVO_IDS, LEADER_SERVO_TYPE, ports[0])
    ctrl2 = ServoController(SERVO_IDS, FOLLOWER_SERVO_TYPE, ports[1])

    with ctrl1, ctrl2:
        # Read voltage from first servo on first port
        voltage1 = ctrl1.read_voltage(SERVO_IDS[0])

        # Leader arm should have voltage ~5V, follower arm ~12V
        if voltage1 < VOLTAGE_THRESHOLD:
            leader, follower = ctrl1, ctrl2
            print(f"{ports[0]}: Leader (voltage: {voltage1:.1f}V)")
            print(f"{ports[1]}: Follower")
        else:
            leader, follower = ctrl2, ctrl1
            print(f"{ports[0]}: Follower (voltage: {voltage1:.1f}V)")
            print(f"{ports[1]}: Leader")

        print(f"\nTeleoperation started at {FREQUENCY}Hz. Ctrl+C to stop.")
        loop_time = 1.0 / FREQUENCY

        try:
            while True:
                start = time.perf_counter()

                # Read leader positions and write to follower
                positions = leader.read_positions()
                follower.write_position(positions)

                # Maintain rate
                elapsed = time.perf_counter() - start
                if elapsed < loop_time:
                    time.sleep(loop_time - elapsed)

        except KeyboardInterrupt:
            print("\nStopped.")


if __name__ == "__main__":
    main()

Found ports: /dev/ttyACM1, /dev/ttyACM2
Checking servo phase values...
Motor 1: Phase already 0 ✓
Motor 2: Phase already 0 ✓
Motor 3: Phase already 0 ✓
Motor 4: Phase already 0 ✓
Motor 5: Phase already 0 ✓
Motor 6: Phase already 0 ✓
Motor 7: Phase already 0 ✓
Checking servo phase values...
Motor 1: Phase already 0 ✓
Motor 2: Phase already 0 ✓
Motor 3: Phase already 0 ✓
Motor 4: Phase already 0 ✓
Motor 5: Phase already 0 ✓
Motor 6: Phase already 0 ✓
Motor 7: Phase already 0 ✓
/dev/ttyACM1: Leader (voltage: 8.4V)
/dev/ttyACM2: Follower

Teleoperation started at 200Hz. Ctrl+C to stop.

Stopped.

Disabling all servos...
Motor 1: Failed to disable - [TxRxResult] Port is in use!
Motor 2: Failed to disable - [TxRxResult] Port is in use!
Motor 3: Failed to disable - [TxRxResult] Port is in use!
Motor 4: Failed to disable - [TxRxResult] Port is in use!
Motor 5: Failed to disable - [TxRxResult] Port is in use!
Motor 6: Failed to disable - [TxRxResult] Port is in use!
Motor 7: Failed to disable -

In [102]:
"""Example of setting servos to middle position (2048)."""

from vassar_feetech_servo_sdk import ServoController
from serial.tools import list_ports
import platform


def main():
    # Define your servo configuration
    servo_ids = [1, 2, 3, 4, 5, 6, 7]
    servo_type = "hls" # sts"  # or "hls" for HLS servos

    try:
        # Auto-detect the servo port
        port = find_servo_port()
        print(f"Found servo port: {port}")

        # Create controller with auto-detected port
        controller = ServoController(servo_ids=servo_ids, servo_type=servo_type, port=port)

        # Connect to servos
        print(f"Connecting to {servo_type.upper()} servos...")
        controller.connect()

    except Exception as e:
        print(f"Error: {e}")
        return

    try:


        # Read current positions before calibration
        print("\n--- Current positions before calibration ---")
        positions_before = controller.read_all_positions()

        for motor_id in sorted(positions_before.keys()):
            pos = positions_before[motor_id]
            diff = pos - 2048
            print(f"Motor {motor_id}: {pos:4d} (off by {diff:+4d} from middle)")

        # Set middle position
        print("\n--- Setting all servos to middle position (2048) ---")
        print("This will calibrate the current physical position as 2048.")
        input("Press Enter to continue (or Ctrl+C to cancel)...")

        success = controller.set_middle_position()

        if success:
            print("\n✓ All servos successfully calibrated to middle position!")

            # Read positions after calibration
            print("\n--- Positions after calibration ---")
            positions_after = controller.read_all_positions()

            for motor_id in sorted(positions_after.keys()):
                pos = positions_after[motor_id]
                print(f"Motor {motor_id}: {pos:4d}")
        else:
            print("\n⚠ Some servos failed to calibrate properly")

    except KeyboardInterrupt:
        print("\nCalibration cancelled")
    except Exception as e:
        print(f"Error: {e}")
    finally:
        controller.disconnect()
        print("\nDisconnected")

def find_servo_port():
    """Find all USB serial ports on the system."""
    print(f"Operating System: {platform.system()}")
    print("\nScanning for USB serial devices...")

    ports = list_ports.comports()
    usb_ports = []

    for port in ports:
        # Filter for likely USB devices
        if platform.system() == "Windows":
            if "COM" in port.device:
                usb_ports.append(port)

    if usb_ports:
        print(f"\nFound {len(usb_ports)} USB serial device(s):")
        for i, port in enumerate(usb_ports):
            print(f"  {i+1}. {port.device} - {port.description}")
            if hasattr(port, 'manufacturer') and port.manufacturer:
                print(f"      Manufacturer: {port.manufacturer}")
            if hasattr(port, 'serial_number') and port.serial_number:
                print(f"      Serial: {port.serial_number}")

        # Return the first port (assuming only 1 device as you mentioned)
        return usb_ports[0].device
    else:
        print("\nNo USB serial devices found!")
        return None


if __name__ == "__main__":
    main()

Operating System: Linux

Scanning for USB serial devices...

No USB serial devices found!
Found servo port: None
Connecting to HLS servos...
Checking servo phase values...
Motor 1: Phase already 0 ✓
Motor 2: Phase already 0 ✓
Motor 3: Phase already 0 ✓
Motor 4: Phase already 0 ✓
Motor 5: Phase already 0 ✓
Motor 6: Phase already 0 ✓
Motor 7: Phase already 0 ✓

--- Current positions before calibration ---
Motor 1: 2046 (off by   -2 from middle)
Motor 2: 2074 (off by  +26 from middle)
Motor 3: 2046 (off by   -2 from middle)
Motor 4: 2048 (off by   +0 from middle)
Motor 5: 2048 (off by   +0 from middle)
Motor 6: 2047 (off by   -1 from middle)
Motor 7: 1720 (off by -328 from middle)

--- Setting all servos to middle position (2048) ---
This will calibrate the current physical position as 2048.

Setting middle position using HLS offset calibration...
Motor 1: Calibrated to 2048 ✓
Motor 2: Calibrated to 2048 ✓
Motor 3: Calibrated to 2048 ✓
Motor 4: Calibrated to 2048 ✓
Motor 5: Calibrated to 